In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests8 as vuong_tests_fast

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params+1]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2


In [3]:

def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


In [4]:
num_sims = 500
trials =1000
adapt_c = True

#calc_c =lambda nobs: (nobs**(1/4)/np.sqrt(nobs), .2*nobs**(1/3))
calc_c =lambda nobs: (nobs**(-1/2)/3, (nobs)**(-5/9)/3) #note divide by 5 works... divide by 3 does not... for c2...



# power 0.... need to see...

In [5]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9


def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [6]:

nobs = 250
calc_c =lambda nobs: (nobs**(-1/2), 3*(nobs)**(-5/9)) #note divide by 5 works... divide by 3 does not... for c2...

c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(c1,c2)
print(mc_out)

Quantiles: 1th: 0.86, 5th: 1.13, 10th: 1.31, 30th: 1.74, 50th: 2.09, 70th: 2.47, 90th: 3.06, 95th: 3.35, 99th: 4.04
bonus term 5.288505625290605 // variance stats 5.599931058923184 //part1 34.903772531645444 //part2 6.599931058923184 5.599931058923184
3.6202820490584315 2.374121495256091 7.027535601334049
-6.956729201522778 2.374121495256091 -3.5494756492471615
---
Quantiles: 1th: 0.83, 5th: 1.16, 10th: 1.36, 30th: 1.85, 50th: 2.24, 70th: 2.66, 90th: 3.34, 95th: 3.75, 99th: 4.48
bonus term 6.246469993774762 // variance stats 4.587759577238115 //part1 34.903772531645444 //part2 5.587759577238116 4.587759577238115
4.714851306287454 1.147690253765027 7.904877318504845
-7.77808868126207 1.147690253765027 -4.58806266904468
---
Quantiles: 1th: 0.77, 5th: 1.06, 10th: 1.25, 30th: 1.68, 50th: 2.02, 70th: 2.38, 90th: 2.96, 95th: 3.24, 99th: 3.89
bonus term 5.9466085069155605 // variance stats 4.869525880349173 //part1 34.903772531645444 //part2 5.869525880349173 4.869525880349173
4.2688616182953

In [7]:

nobs = 250
calc_c =lambda nobs: (nobs**(-1/2)/3, (nobs)**(-5/9)/3) #note divide by 5 works... divide by 3 does not... for c2...

c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(c1,c2)
print(mc_out)

Quantiles: 1th: 0.81, 5th: 1.09, 10th: 1.26, 30th: 1.71, 50th: 2.06, 70th: 2.42, 90th: 2.99, 95th: 3.27, 99th: 3.85
bonus term 0.8122947602261299 // variance stats 4.441038160680707 //part1 3.8781969479606047 //part2 4.774371494014041 4.441038160680707
-0.9316588893263341 1.1047155322390805 2.5982500747011286
-2.5562484097785942 1.1047155322390805 0.9736605542488691
---
Quantiles: 1th: 0.78, 5th: 1.10, 10th: 1.29, 30th: 1.75, 50th: 2.09, 70th: 2.46, 90th: 3.08, 95th: 3.38, 99th: 3.95
bonus term 0.8884624595930234 // variance stats 4.0317322989693345 //part1 3.8781969479606047 //part2 4.365065632302668 4.0317322989693345
-1.0606990770483526 1.8884478613251474 2.7673326187295992
-2.837623996234399 1.8884478613251474 0.9904076995435527
---
Quantiles: 1th: 0.76, 5th: 1.07, 10th: 1.24, 30th: 1.66, 50th: 2.01, 70th: 2.37, 90th: 2.94, 95th: 3.23, 99th: 3.77
bonus term 0.8012992889956647 // variance stats 4.5065523388747035 //part1 3.8781969479606047 //part2 4.839885672208037 4.506552338874703

In [8]:

nobs = 250
calc_c =lambda nobs: (nobs**(-1/2), (nobs)**(-5/9)/3) #note divide by 5 works... divide by 3 does not... for c2...

c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(c1,c2)
print(mc_out)

Quantiles: 1th: 0.84, 5th: 1.14, 10th: 1.34, 30th: 1.79, 50th: 2.16, 70th: 2.54, 90th: 3.14, 95th: 3.44, 99th: 4.10
bonus term 0.6734444260587534 // variance stats 4.75874830631126 //part1 3.8781969479606047 //part2 5.75874830631126 4.75874830631126
-0.9220871988631256 0.7608079386949215 2.2154491817294777
-2.2689760509806325 0.7608079386949215 0.8685603296119708
---
Quantiles: 1th: 0.84, 5th: 1.19, 10th: 1.38, 30th: 1.86, 50th: 2.24, 70th: 2.65, 90th: 3.28, 95th: 3.62, 99th: 4.25
bonus term 0.7499739940273641 // variance stats 4.1711085702247725 //part1 3.8781969479606047 //part2 5.1711085702247725 4.1711085702247725
-0.8843390450907134 2.126855272299043 2.370250377274331
-2.3842870331454415 2.126855272299043 0.8703023892196028
---
Quantiles: 1th: 0.92, 5th: 1.23, 10th: 1.44, 30th: 1.91, 50th: 2.30, 70th: 2.72, 90th: 3.38, 95th: 3.68, 99th: 4.32
bonus term 0.582069097346738 // variance stats 5.662777607742275 //part1 3.8781969479606047 //part2 6.662777607742275 5.662777607742275
-1.07

In [9]:

nobs = 250
calc_c =lambda nobs: (nobs**(-1/2), (nobs)**(-5/9)/5) #note divide by 5 works... divide by 3 does not... for c2...

c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(c1,c2)
print(mc_out)

Quantiles: 1th: 0.89, 5th: 1.19, 10th: 1.38, 30th: 1.81, 50th: 2.18, 70th: 2.59, 90th: 3.19, 95th: 3.53, 99th: 4.06
bonus term 0.35451942426540844 // variance stats 5.563584417406511 //part1 2.3269181687763627 //part2 6.563584417406511 5.563584417406511
-1.2320299450786523 2.347932211117526 2.0817433815978945
-1.941068793609469 2.347932211117526 1.3727045330670777
---
Quantiles: 1th: 0.84, 5th: 1.11, 10th: 1.32, 30th: 1.79, 50th: 2.18, 70th: 2.58, 90th: 3.20, 95th: 3.53, 99th: 4.10
bonus term 0.4416100709390753 // variance stats 4.269169165069574 //part1 2.3269181687763627 //part2 5.269169165069574 4.269169165069574
-1.1634413222637445 0.33784922980052184 2.1263612051175405
-2.046661464141895 0.33784922980052184 1.2431410632393902
---
Quantiles: 1th: 0.86, 5th: 1.14, 10th: 1.34, 30th: 1.82, 50th: 2.17, 70th: 2.56, 90th: 3.19, 95th: 3.47, 99th: 4.12
bonus term 0.402887463585302 // variance stats 4.775603311329374 //part1 2.3269181687763627 //part2 5.775603311329374 4.775603311329374
-1.

In [10]:

nobs = 250
calc_c =lambda nobs: (nobs**(-1/2)/3, (nobs)**(-5/9)/5) #note divide by 5 works... divide by 3 does not... for c2...

c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(c1,c2)
print(mc_out)

Quantiles: 1th: 0.93, 5th: 1.20, 10th: 1.39, 30th: 1.86, 50th: 2.22, 70th: 2.61, 90th: 3.25, 95th: 3.56, 99th: 4.25
bonus term 0.3740565953784843 // variance stats 5.887431645885655 //part1 2.3269181687763627 //part2 6.2207649792189885 5.887431645885655
-1.3939472384917584 1.5851594348904778 2.2953333224228065
-2.1420604292487266 1.5851594348904778 1.5472201316658383
---
Quantiles: 1th: 0.85, 5th: 1.16, 10th: 1.33, 30th: 1.77, 50th: 2.11, 70th: 2.50, 90th: 3.10, 95th: 3.41, 99th: 4.06
bonus term 0.3730464079865524 // variance stats 5.904277105902536 //part1 2.3269181687763627 //part2 6.237610439235869 5.904277105902536
-1.6316654198085585 2.185152000473114 2.537848464454141
-2.3777582357816636 2.185152000473114 1.7917556484810357
---
Quantiles: 1th: 0.85, 5th: 1.13, 10th: 1.33, 30th: 1.75, 50th: 2.10, 70th: 2.49, 90th: 3.09, 95th: 3.42, 99th: 4.12
bonus term 0.3873693746927865 // variance stats 5.673641716312205 //part1 2.3269181687763627 //part2 6.006975049645539 5.673641716312205
-1.

# a  = .25, k= 9

In [11]:
a = 0.25
num_params=9



In [12]:
nobs = 250
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

Quantiles: 1th: 0.81, 5th: 1.15, 10th: 1.33, 30th: 1.78, 50th: 2.15, 70th: 2.54, 90th: 3.10, 95th: 3.40, 99th: 3.93
bonus term 0.4419407757818971 // variance stats 4.931892905769626 //part1 2.3269181687763627 //part2 5.26522623910296 4.931892905769626
-1.3833574685728365 1.0013063651672527 2.494657153442089
-2.2672390201366306 1.0013063651672527 1.6107756018782944
---
Quantiles: 1th: 0.80, 5th: 1.10, 10th: 1.31, 30th: 1.77, 50th: 2.16, 70th: 2.57, 90th: 3.19, 95th: 3.49, 99th: 4.16
bonus term 0.5076122658423712 // variance stats 4.250713044889805 //part1 2.3269181687763627 //part2 4.584046378223138 4.250713044889805
-1.2181813190636783 0.3511746704871375 2.281246452755066
-2.2334058507484205 0.3511746704871375 1.2660219210703239
---
Quantiles: 1th: 0.75, 5th: 1.02, 10th: 1.20, 30th: 1.58, 50th: 1.91, 70th: 2.27, 90th: 2.83, 95th: 3.12, 99th: 3.70
bonus term 0.3949221027779369 // variance stats 5.558760708525801 //part1 2.3269181687763627 //part2 5.892094041859134 5.558760708525801
-1.5

In [13]:
nobs = 500
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

Quantiles: 1th: 0.75, 5th: 1.05, 10th: 1.22, 30th: 1.63, 50th: 1.96, 70th: 2.31, 90th: 2.87, 95th: 3.13, 99th: 3.71
bonus term 0.5057838842856659 // variance stats 5.927140900176535 //part1 3.1664469752949476 //part2 6.260474233509868 5.927140900176535
-1.3070009313654005 1.6761120699409973 2.596656719627914
-2.318568699936732 1.6761120699409973 1.5850889510565822
---
Quantiles: 1th: 0.87, 5th: 1.11, 10th: 1.30, 30th: 1.72, 50th: 2.05, 70th: 2.41, 90th: 2.98, 95th: 3.24, 99th: 3.77
bonus term 0.4138116668358703 // variance stats 7.318570247603771 //part1 3.1664469752949476 //part2 7.651903580937104 7.318570247603771
-1.3618123564037163 1.990800017433336 2.25627924191759
-2.189435690075457 1.990800017433336 1.4286559082458496
---
Quantiles: 1th: 0.84, 5th: 1.16, 10th: 1.35, 30th: 1.82, 50th: 2.18, 70th: 2.58, 90th: 3.20, 95th: 3.52, 99th: 4.11
bonus term 0.4733714642349255 // variance stats 6.355804509845779 //part1 3.1664469752949476 //part2 6.689137843179112 6.355804509845779
-1.37539

In [14]:
nobs = 100
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

Quantiles: 1th: 0.91, 5th: 1.22, 10th: 1.44, 30th: 1.90, 50th: 2.30, 70th: 2.74, 90th: 3.47, 95th: 3.84, 99th: 4.60
bonus term 0.3698385768366859 // variance stats 3.8537024772821065 //part1 1.5485273653622538 //part2 4.1870358106154395 3.8537024772821065
-1.5294127748081496 0.1300304025899361 2.2530754604936045
-2.269089928481521 0.1300304025899361 1.5133983068202326
---
Quantiles: 1th: 0.78, 5th: 1.08, 10th: 1.26, 30th: 1.70, 50th: 2.05, 70th: 2.42, 90th: 3.03, 95th: 3.31, 99th: 3.96
bonus term 0.409681502374352 // variance stats 3.4464989421968077 //part1 1.5485273653622538 //part2 3.7798322755301412 3.4464989421968077
-1.3200471662584201 0.18653819172151648 2.4402013389526744
-2.139410171007124 0.18653819172151648 1.6208383342039703
---
Quantiles: 1th: 0.78, 5th: 1.11, 10th: 1.33, 30th: 1.80, 50th: 2.23, 70th: 2.76, 90th: 3.69, 95th: 4.26, 99th: 5.38
bonus term 0.44247956584328074 // variance stats 3.166325147869204 //part1 1.5485273653622538 //part2 3.499658481202537 3.16632514786

# a  = .25, k= 4, something not right?

In [15]:
a = 0.25
num_params=4



In [16]:

nobs = 250
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=False)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.97 & 0.97 & 0.97 & 0.97 & 0.99 & 0.99   \\
Model 1 & 0.01 & 0.01 & 0.01 & 0.02 & 0.00 & 0.00   \\
Model 2 & 0.03 & 0.03 & 0.02 & 0.01 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.968, 0.006, 0.026]), array([0.968, 0.006, 0.026]), array([0.966, 0.01 , 0.024]), array([0.966, 0.02 , 0.014]), array([0.99 , 0.004, 0.006]), array([0.994, 0.002, 0.004]), -1.2001347544444618, 5.5490481240682294, 5.613545135885435)


# a  = .25, K = 19

In [17]:
a = 0.25
num_params=19


In [18]:

nobs = 250
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=False)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.88 & 0.88 & 0.90 & 0.81 & 0.94 & 0.95   \\
Model 1 & 0.12 & 0.12 & 0.10 & 0.19 & 0.06 & 0.05   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.88, 0.12, 0.  ]), array([0.88, 0.12, 0.  ]), array([0.896, 0.104, 0.   ]), array([0.81, 0.19, 0.  ]), array([0.936, 0.064, 0.   ]), array([0.948, 0.052, 0.   ]), 4.052067124187434, 4.6771873922293175, 4.475289137394027)


# evidence of power

In [19]:

nobs = 500
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=False)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.58 & 0.58 & 0.59 & 0.41 & 0.75 & 0.72   \\
Model 1 & 0.42 & 0.42 & 0.41 & 0.59 & 0.25 & 0.28   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.576, 0.424, 0.   ]), array([0.576, 0.424, 0.   ]), array([0.588, 0.412, 0.   ]), array([0.412, 0.588, 0.   ]), array([0.746, 0.254, 0.   ]), array([0.718, 0.282, 0.   ]), 9.388282926581237, 5.133262031935886, 4.9747100361033185)


In [20]:

nobs = 100
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=False)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.48 & 0.51 & 0.57 & 0.41 & 0.58 & 0.69   \\
Model 1 & 0.52 & 0.49 & 0.43 & 0.59 & 0.42 & 0.31   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.484, 0.516, 0.   ]), array([0.51, 0.49, 0.  ]), array([0.568, 0.432, 0.   ]), array([0.408, 0.592, 0.   ]), array([0.578, 0.422, 0.   ]), array([0.686, 0.314, 0.   ]), 9.557569717192756, 4.963639391998627, 4.712630474252887)


In [21]:

nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=False)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.55 & 0.55 & 0.60 & 0.44 & 0.70 & 0.73   \\
Model 1 & 0.45 & 0.45 & 0.40 & 0.56 & 0.30 & 0.27   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.548, 0.452, 0.   ]), array([0.554, 0.446, 0.   ]), array([0.604, 0.396, 0.   ]), array([0.442, 0.558, 0.   ]), array([0.704, 0.296, 0.   ]), array([0.732, 0.268, 0.   ]), 9.307486472429229, 4.878859799604461, 4.854934177382362)


In [22]:

nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=False)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.57 & 0.57 & 0.61 & 0.43 & 0.69 & 0.72   \\
Model 1 & 0.43 & 0.43 & 0.39 & 0.57 & 0.31 & 0.28   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.57, 0.43, 0.  ]), array([0.574, 0.426, 0.   ]), array([0.614, 0.386, 0.   ]), array([0.43, 0.57, 0.  ]), array([0.692, 0.308, 0.   ]), array([0.724, 0.276, 0.   ]), 9.310374269503196, 5.132228616531518, 4.863789190962079)


# evidence of power 2

In [23]:
nobs = 250
c1,c2 = calc_c(nobs)



a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=False)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.92 & 0.92 & 0.93 & 0.98 & 1.00 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.08 & 0.08 & 0.07 & 0.02 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.918, 0.   , 0.082]), array([0.922, 0.   , 0.078]), array([0.93, 0.  , 0.07]), array([0.982, 0.002, 0.016]), array([0.998, 0.   , 0.002]), array([1., 0., 0.]), -3.6089904530209447, 2.5666529653795935, 2.873680694153721)


In [24]:
nobs = 500
c1,c2 = calc_c(nobs)



a2,a1 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=False)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.91 & 0.91 & 0.92 & 0.97 & 1.00 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.09 & 0.09 & 0.08 & 0.03 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.91, 0.  , 0.09]), array([0.91, 0.  , 0.09]), array([0.916, 0.   , 0.084]), array([0.97, 0.  , 0.03]), array([0.998, 0.   , 0.002]), array([0.998, 0.   , 0.002]), -3.773423219879391, 2.850870953391144, 2.9399257598455137)


In [25]:
nobs = 100
c1,c2 = calc_c(nobs)



a2,a1 =np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=False)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.91 & 0.94 & 0.94 & 0.98 & 0.99 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.09 & 0.06 & 0.06 & 0.02 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.91, 0.  , 0.09]), array([0.936, 0.   , 0.064]), array([0.938, 0.   , 0.062]), array([0.976, 0.002, 0.022]), array([0.994, 0.   , 0.006]), array([0.998, 0.   , 0.002]), -3.571637821303764, 2.4182878859348462, 2.767963369885053)


In [26]:
nobs = 250
c1,c2 = calc_c(nobs)


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=False)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.18 & 0.19 & 0.24 & 0.42 & 0.69 & 0.72   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.82 & 0.81 & 0.76 & 0.58 & 0.31 & 0.28   \\
\hline
\end{tabular}
(array([0.184, 0.   , 0.816]), array([0.192, 0.   , 0.808]), array([0.238, 0.   , 0.762]), array([0.418, 0.   , 0.582]), array([0.694, 0.   , 0.306]), array([0.718, 0.   , 0.282]), -12.735658862388457, 5.077045730865144, 4.97172505532564)


In [27]:
nobs = 250
c1,c2 = calc_c(nobs)



a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=False)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.93 & 0.94 & 0.94 & 0.98 & 1.00 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.07 & 0.06 & 0.06 & 0.02 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.932, 0.   , 0.068]), array([0.936, 0.   , 0.064]), array([0.936, 0.   , 0.064]), array([0.98 , 0.004, 0.016]), array([0.996, 0.   , 0.004]), array([0.996, 0.   , 0.004]), -3.6361470967988003, 2.5977960298733334, 2.8842612496689917)
